[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA3/blob/main/lab9/lab9.ipynb)

# Práctica 9: One Class Network

## Pre-requisitos

### Instalar paquetes

Si la práctica requiere algún paquete de Python, habrá que incluir una celda en la que se instalen. Si usamos un paquete que se ha utilizado en prácticas anteriores, podríamos dar por supuesto que está instalado pero no cuesta nada satisfacer todas las dependencias en la propia práctica para reducir las dependencias entre ellas.

### NOTA: En <font color='red'>Google Colab</font> hay que instalar los paquetes EN CADA EJECUCIÓN

In [ ]:
# Ejemplo de instalación de tensorflow 2.0
#%tensorflow_version 2.x
# !pip3 install tensorflow  # NECESARIO SOLO SI SE EJECUTA EN LOCAL
import tensorflow as tf

# Hacemos los imports que sean necesarios
import numpy as np

# One Class sobre datos artificiales

Lo primero que tenemos que hacer es definir los datos a utilizar.

In [ ]:
random_state = 42
rng = np.random.RandomState(random_state)
#  datos de entrenamiento
X = 0.3 * rng.randn(5000, 2)
x_train = np.r_[X + 2, X - 2]
#  datos de test en la misma distribución que los datos de entrenamiento
X = 0.3 * rng.randn(200, 2)
x_test = np.r_[X + 2, X - 2]
#  outliers
x_outliers = rng.uniform(low=-4, high=4, size=(200, 2))

## Crea tu propia red para la detección de anomalías

Vamos a crear nuestra propia red para la detección de anomalías. Para ello, vamos a definir una red cualquiera, que nos **transforme los datos de entrada en una salida de un único elemento**. Esta red va a cumplir una serie de características:

* La capa anterior a la salida serán las llamadas **deep features**.
* Todas las capas (incluyendo la última) deben incluir regularización.
* La función de coste es $$L(y, \tilde{y}) = \dfrac{1}{2} \| w^2 \| + \dfrac{1}{\nu} \dfrac{1}{N} \sum_{i=1}^N \max(0, r - \tilde{y}) $$ donde $\tilde{y}$ es la salida de la red, $\nu$ es un hiperparámetro entre 0 y 1, y $r$ es un parámetro no entrenable, pero que va a ser modificado en cada epoch.
* Al final del cada epoch, r va a ser modificado al valor del $\nu$-cuantil de los datos de entrada (este valor será modificado gracias al Callback proporcionado a continuación).
* Para la predicción, se considerará un dato típico si $\tilde{y} > r$. En caso contrario, será un dato atípico.

In [ ]:
class ChangeRCallback(tf.keras.callbacks.Callback):
   def __init__(self, train_data, delta=.025, steps=3):
       super().__init__()
       self.train_data = train_data
       self.delta = delta
       self.steps = steps
       self.cont = 0

   def on_epoch_end(self, epoch, logs=None):
       sorted_values = np.sort(self.model.predict(self.train_data).flatten())
       new_value = sorted_values[int(len(sorted_values) * (1. - self.model.nu))]
       old_value = self.model.r.numpy()
       print('Cambiando r a', new_value, ', max:', sorted_values.max(), ', min:', sorted_values.min())
       self.model.r.assign(new_value)
       if np.abs(old_value - new_value) < self.delta:
            self.cont += 1
            if self.cont >= self.steps:
                print('Convergencia obtenida. Finalizando el entrenamiento.')
                self.model.stop_training = True
       else:
            self.cont = 0

Tu trabajo es crear el modelo y entrenarlo.

In [ ]:
# TODO: implementa la red de detección de anomalías

class DetectorAnomalias:

    def __init__(self, input_shape, nu=.5):
        # TODO : define el modelo

        self.model = None

        self.model.r = tf.Variable(1.0, trainable=False, name='r', dtype=tf.float32)
        self.model.nu = tf.Variable(nu, trainable=False, name='nu', dtype=tf.float32)
        
      
    def loss_function(self, y_true, y_pred):
        # TODO: crea la función de pérdida
        None
    
    def fit(self, X, y=None, sample_weight=None):
        # TODO: entrena el modelo. Escoge el tamaño de batch y el número de epochs que quieras. No te olvides del callback.
        dummy_y = np.zeros((len(X), 1)) # Necesario pasar como salida para que keras no de un error
        None
        
    def predict(self, X):
        # TODO: Devuelve la predicción del modelo
        None

    def get_encoded_data(self, X):
        # TODO: devuelve la salida del encoder (code)
        None
        
    def __del__(self):
        # TODO: borra el modelo
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

### Entrena el modelo.

Usa lo hecho anteriormente para entrenar tu modelo.

In [ ]:
# TODO: Define el modelo


In [ ]:
# TODO: Entrena tu modelo


## Evaluando el modelo

Una vez entrenado, para evaluar el modelo sólo hay que tener en cuenta lo siguiente:

  1. Si la salida es positiva, es un dato típico.
  1. Si la salida es negativa, es un dato atípico.

### TRABAJO: Evalúa el modelo con los datos del conjunto de test, y con los outliers. Visualiza los datos típicos y atípicos con una gráfica.

In [ ]:
# TODO: Evalúa el modelo con los datos del conjunto de test. Indica el porcentaje de datos etiquetados como típicos, y visualiza los datos


In [ ]:
# TODO: Evalúa el modelo con los datos del conjunto de outliers. Indica el porcentaje de datos etiquetados como atípicos, y visualiza los datos en conjunto con los de test


¿Qué resultados has obtenido? Si el número de outliers detectado es bajo (inferior al 30%), puedes estar cometiendo algún error, entre ellos:

* Sobreentrenar el modelo. Prueba a usar un delta distinto en el callback.
* Usar un valor de $\nu$ demasiado alto.

Prueba distintas configuraciones para ver su efecto.

# ¡ENHORABUENA! Has completado la práctica de oneclass.
